In [1]:
import numpy as np
import pandas as pd
import cv2
import dlib
import os
import face_recognition
from PIL import Image

In [ ]:
# replace this image directory with yours

path = './CACD2000.tar/CACD2000' 

# directory path
orig_path = path

# folder to store the image
out_path = os.path.join(path, 'CACD-centered')


if not os.path.exists(orig_path):
    raise ValueError(f'Original image path {orig_path} does not exist.')

if not os.path.exists(out_path):
    os.mkdir(out_path)





# looping through the images in your image directory
# I'm just taking only one image from the image directory
# here
for picture_name in os.listdir(orig_path)[:-1]:
    img = cv2.imread(os.path.join(orig_path, picture_name))
    #print(img.shape)
    #cv2.imshow('window11', img)
    

    #fiding face locations
    print(picture_name)
    faces=face_recognition.face_locations(img)
    
    #if len(faces) !=1:
    #        shutil.move(os.path.join(orig_path, picture_name), './CACD2000.tar/deleted')
    #iterating each face
    for face in faces:
        #file name with face location
        if len(faces) !=1:
            continue
        
        t = face[0]  # top location
        r = face[1]  # right location
        b = face[2]  # bottom location
        l = face[3]  # left location


        # just checking Raschka code from here
        width = r - l # width is right minus left
        height = b - t # height is bottom minus top
        diff =  height-width
        tol = 15 # additional increment
        up_down = 5

                
        if(diff > 0):#if height is greater than width -> adding access dimension left and right and moving up by 5 + tol
            if not diff % 2:  # if symmetric
                t=t-tol-up_down 
                b=b+tol-up_down
                l=l-tol-int(diff/2)
                r=r+tol+int(diff/2)
                tmp = img[(t):(b),(l):(r),:]
            else:
                t=t-tol-up_down
                b=b+tol-up_down
                l=l-tol-int((diff-1)/2)
                r=r+tol+int((diff+1)/2)
                tmp = img[(t):(b),(l):(r),:]
        if(diff <= 0):#if width is greater than hight -> adding access dimension to up and down and moving up by 5 +tol
            if not diff % 2:  # symmetric
                t=t-tol-int(diff/2)-up_down
                b=b+tol+int(diff/2)-up_down
                l=l-tol
                r=r+tol
                tmp = img[(t):(b),(l):(r),:]
            else:
                t=t-tol-int((diff-1)/2)-up_down
                b=b+tol+int((diff+1)/2)-up_down
                l=l-tol
                r=r+tol
                tmp = img[(t):(b),(l):(r),:]
    
        
        
        #cv2.rectangle(img, (l, t), (r, b), (0,255,0), 2)
        
        
        #save only face 
        try:
            tmp = np.array(Image.fromarray(np.uint8(tmp)).resize((120, 120), Image.ANTIALIAS))
            #A UINT8 is an 8-bit unsigned integer (range: 0 through 255 decimal). 
            #Because a UINT8 is unsigned, its first bit (Most Significant Bit (MSB)) is not reserved for signing
            
            #anti-aliasing is a technique for minimizing the distortion artifacts known as aliasing 
            #when representing a high-resolution image at a lower resolution
            
            #This function converts a numerical (integer or float) numpy array of any size and dimensionality into a image
            cv2.imwrite(os.path.join(out_path, picture_name), tmp)
            print(f'Wrote {picture_name}')
            keep_picture.append(picture_name)
        except ValueError:
            print(f'Failed {picture_name}')
            pass    
        #if len(faces) ==1:
        #   shutil.move(os.path.join(orig_path, picture_name), './CACD2000.tar/processed') 
        

#287 218 349 281 1ltrb  trbl

32_Sonya_Walger_0001.jpg
Wrote 32_Sonya_Walger_0001.jpg
32_Sonya_Walger_0002.jpg
Wrote 32_Sonya_Walger_0002.jpg
32_Sonya_Walger_0004.jpg
Wrote 32_Sonya_Walger_0004.jpg
32_Sonya_Walger_0009.jpg
Wrote 32_Sonya_Walger_0009.jpg
32_Sonya_Walger_0012.jpg
Wrote 32_Sonya_Walger_0012.jpg
32_Sonya_Walger_0015.jpg
Wrote 32_Sonya_Walger_0015.jpg
32_Sonya_Walger_0016.jpg
Wrote 32_Sonya_Walger_0016.jpg
32_Staci_Keanan_0002.jpg
Wrote 32_Staci_Keanan_0002.jpg
32_Staci_Keanan_0010.jpg
Wrote 32_Staci_Keanan_0010.jpg
32_Staci_Keanan_0011.jpg
Wrote 32_Staci_Keanan_0011.jpg
32_Stacy_Keibler_0001.jpg
Wrote 32_Stacy_Keibler_0001.jpg
32_Stacy_Keibler_0002.jpg
Wrote 32_Stacy_Keibler_0002.jpg
32_Stacy_Keibler_0003.jpg
Wrote 32_Stacy_Keibler_0003.jpg
32_Stacy_Keibler_0004.jpg
Wrote 32_Stacy_Keibler_0004.jpg
32_Stacy_Keibler_0005.jpg
Wrote 32_Stacy_Keibler_0005.jpg
32_Stacy_Keibler_0006.jpg
Wrote 32_Stacy_Keibler_0006.jpg
32_Stacy_Keibler_0007.jpg
Wrote 32_Stacy_Keibler_0007.jpg
32_Stacy_Keibler_0008.jpg
Wrote 32

Wrote 32_Taylor_Kitsch_0005.jpg
32_Taylor_Kitsch_0006.jpg
Wrote 32_Taylor_Kitsch_0006.jpg
32_Taylor_Kitsch_0007.jpg
Wrote 32_Taylor_Kitsch_0007.jpg
32_Taylor_Kitsch_0008.jpg
Wrote 32_Taylor_Kitsch_0008.jpg
32_Taylor_Kitsch_0010.jpg
Wrote 32_Taylor_Kitsch_0010.jpg
32_Taylor_Kitsch_0011.jpg
Wrote 32_Taylor_Kitsch_0011.jpg
32_Taylor_Kitsch_0012.jpg
Wrote 32_Taylor_Kitsch_0012.jpg
32_Taylor_Kitsch_0013.jpg
Wrote 32_Taylor_Kitsch_0013.jpg
32_Taylor_Kitsch_0014.jpg
Wrote 32_Taylor_Kitsch_0014.jpg
32_Taylor_Kitsch_0015.jpg
Wrote 32_Taylor_Kitsch_0015.jpg
32_Taylor_Kitsch_0016.jpg
Wrote 32_Taylor_Kitsch_0016.jpg
32_Taylor_Kitsch_0017.jpg
Wrote 32_Taylor_Kitsch_0017.jpg
32_Taylor_Kitsch_0018.jpg
Wrote 32_Taylor_Kitsch_0018.jpg
32_Taylor_Kitsch_0019.jpg
Wrote 32_Taylor_Kitsch_0019.jpg
32_Taylor_Kitsch_0020.jpg
Wrote 32_Taylor_Kitsch_0020.jpg
32_Thandie_Newton_0001.jpg
Wrote 32_Thandie_Newton_0001.jpg
32_Thandie_Newton_0002.jpg
Wrote 32_Thandie_Newton_0002.jpg
32_Thandie_Newton_0003.jpg
Wrote 32_

Wrote 32_Vinessa_Shaw_0007.jpg
32_Vinessa_Shaw_0008.jpg
Wrote 32_Vinessa_Shaw_0008.jpg
32_Vinessa_Shaw_0009.jpg
Wrote 32_Vinessa_Shaw_0009.jpg
32_Vinessa_Shaw_0010.jpg
Wrote 32_Vinessa_Shaw_0010.jpg
32_Vinessa_Shaw_0011.jpg
Wrote 32_Vinessa_Shaw_0011.jpg
32_Vinessa_Shaw_0012.jpg
Wrote 32_Vinessa_Shaw_0012.jpg
32_Warren_Kole_0005.jpg
Wrote 32_Warren_Kole_0005.jpg
32_Wentworth_Miller_0003.jpg
Wrote 32_Wentworth_Miller_0003.jpg
32_Wentworth_Miller_0004.jpg
Wrote 32_Wentworth_Miller_0004.jpg
32_Wentworth_Miller_0007.jpg
Wrote 32_Wentworth_Miller_0007.jpg
32_Wentworth_Miller_0009.jpg
Wrote 32_Wentworth_Miller_0009.jpg
32_Wentworth_Miller_0011.jpg
Wrote 32_Wentworth_Miller_0011.jpg
32_Wentworth_Miller_0016.jpg
Wrote 32_Wentworth_Miller_0016.jpg
32_Wentworth_Miller_0018.jpg
Wrote 32_Wentworth_Miller_0018.jpg
32_Wil_Wheaton_0001.jpg
32_Wil_Wheaton_0006.jpg
Wrote 32_Wil_Wheaton_0006.jpg
32_Wil_Wheaton_0007.jpg
Wrote 32_Wil_Wheaton_0007.jpg
32_Wil_Wheaton_0008.jpg
Wrote 32_Wil_Wheaton_0008.jpg
3

Wrote 33_Aishwarya_Rai_Bachchan_0014.jpg
33_Aishwarya_Rai_Bachchan_0016.jpg
Wrote 33_Aishwarya_Rai_Bachchan_0016.jpg
33_Aishwarya_Rai_Bachchan_0017.jpg
Wrote 33_Aishwarya_Rai_Bachchan_0017.jpg
33_Aishwarya_Rai_Bachchan_0019.jpg
Wrote 33_Aishwarya_Rai_Bachchan_0019.jpg
33_Alanna_Ubach_0001.jpg
Wrote 33_Alanna_Ubach_0001.jpg
33_Alanna_Ubach_0002.jpg
Wrote 33_Alanna_Ubach_0002.jpg
33_Alanna_Ubach_0003.jpg
Wrote 33_Alanna_Ubach_0003.jpg
33_Alanna_Ubach_0012.jpg
Wrote 33_Alanna_Ubach_0012.jpg
33_Alanna_Ubach_0013.jpg
Wrote 33_Alanna_Ubach_0013.jpg
33_Alanna_Ubach_0014.jpg
Wrote 33_Alanna_Ubach_0014.jpg
33_Alanna_Ubach_0015.jpg
Wrote 33_Alanna_Ubach_0015.jpg
33_Alanna_Ubach_0017.jpg
Wrote 33_Alanna_Ubach_0017.jpg
33_Alan_Tudyk_0002.jpg
Wrote 33_Alan_Tudyk_0002.jpg
33_Alexandra_Maria_Lara_0003.jpg
Wrote 33_Alexandra_Maria_Lara_0003.jpg
33_Alexandra_Maria_Lara_0006.jpg
Wrote 33_Alexandra_Maria_Lara_0006.jpg
33_Alexandra_Maria_Lara_0007.jpg
Wrote 33_Alexandra_Maria_Lara_0007.jpg
33_Alex_O'Lough

Wrote 33_Angelina_Jolie_0007.jpg
33_Angelina_Jolie_0008.jpg
Wrote 33_Angelina_Jolie_0008.jpg
33_Angelina_Jolie_0009.jpg
Wrote 33_Angelina_Jolie_0009.jpg
33_Angelina_Jolie_0010.jpg
Wrote 33_Angelina_Jolie_0010.jpg
33_Angelina_Jolie_0011.jpg
Wrote 33_Angelina_Jolie_0011.jpg
33_Angelina_Jolie_0012.jpg
Wrote 33_Angelina_Jolie_0012.jpg
33_Angelina_Jolie_0013.jpg
Wrote 33_Angelina_Jolie_0013.jpg
33_Angelina_Jolie_0014.jpg
Wrote 33_Angelina_Jolie_0014.jpg
33_Angelina_Jolie_0015.jpg
Wrote 33_Angelina_Jolie_0015.jpg
33_Angelina_Jolie_0016.jpg
Wrote 33_Angelina_Jolie_0016.jpg
33_Angelina_Jolie_0017.jpg
Wrote 33_Angelina_Jolie_0017.jpg
33_Angie_Harmon_0001.jpg
Wrote 33_Angie_Harmon_0001.jpg
33_Angie_Harmon_0002.jpg
Wrote 33_Angie_Harmon_0002.jpg
33_Angie_Harmon_0003.jpg
Wrote 33_Angie_Harmon_0003.jpg
33_Angie_Harmon_0004.jpg
Wrote 33_Angie_Harmon_0004.jpg
33_Angie_Harmon_0005.jpg
Wrote 33_Angie_Harmon_0005.jpg
33_Angie_Harmon_0009.jpg
Wrote 33_Angie_Harmon_0009.jpg
33_Angie_Harmon_0010.jpg
Wrote 

...

34_Colin_Hanks_0009.jpg
Wrote 34_Colin_Hanks_0009.jpg
34_Colin_Hanks_0010.jpg
Wrote 34_Colin_Hanks_0010.jpg
34_Colin_Hanks_0011.jpg
Wrote 34_Colin_Hanks_0011.jpg
34_Colin_Hanks_0012.jpg
Wrote 34_Colin_Hanks_0012.jpg
34_Colin_Hanks_0014.jpg
Wrote 34_Colin_Hanks_0014.jpg
34_Common_0002.jpg
Wrote 34_Common_0002.jpg
34_Common_0004.jpg
Wrote 34_Common_0004.jpg
34_Common_0005.jpg
Wrote 34_Common_0005.jpg
34_Common_0008.jpg
Wrote 34_Common_0008.jpg
34_Common_0009.jpg
34_Common_0010.jpg
Wrote 34_Common_0010.jpg
34_Common_0012.jpg
Wrote 34_Common_0012.jpg
34_Common_0013.jpg
Wrote 34_Common_0013.jpg
34_Common_0014.jpg
Wrote 34_Common_0014.jpg
34_Corey_Feldman_0001.jpg
Wrote 34_Corey_Feldman_0001.jpg
34_Corey_Feldman_0003.jpg
Wrote 34_Corey_Feldman_0003.jpg
34_Corey_Feldman_0006.jpg
Wrote 34_Corey_Feldman_0006.jpg
34_Corey_Feldman_0007.jpg
Wrote 34_Corey_Feldman_0007.jpg
34_Corey_Feldman_0010.jpg
Wrote 34_Corey_Feldman_0010.jpg
34_Corey_Feldman_0011.jpg
Wrote 34_Corey_Feldman_0011.jpg
34_Corey_Fe

Wrote 34_David_Tennant_0008.jpg
34_David_Tennant_0010.jpg
Wrote 34_David_Tennant_0010.jpg
34_David_Tennant_0013.jpg
34_David_Tennant_0014.jpg
Wrote 34_David_Tennant_0014.jpg
34_David_Tennant_0015.jpg
Wrote 34_David_Tennant_0015.jpg
34_David_Tennant_0016.jpg
Wrote 34_David_Tennant_0016.jpg
34_Dax_Shepard_0002.jpg
Wrote 34_Dax_Shepard_0002.jpg
34_Dax_Shepard_0003.jpg
Wrote 34_Dax_Shepard_0003.jpg
34_Dax_Shepard_0004.jpg
Wrote 34_Dax_Shepard_0004.jpg
34_Dax_Shepard_0006.jpg
Wrote 34_Dax_Shepard_0006.jpg
34_Denise_Richards_0001.jpg
34_Denise_Richards_0002.jpg
Wrote 34_Denise_Richards_0002.jpg
34_Denise_Richards_0003.jpg
Wrote 34_Denise_Richards_0003.jpg
34_Denise_Richards_0004.jpg
Wrote 34_Denise_Richards_0004.jpg
34_Denise_Richards_0005.jpg
Wrote 34_Denise_Richards_0005.jpg
34_Denise_Richards_0006.jpg
Wrote 34_Denise_Richards_0006.jpg
34_Denise_Richards_0009.jpg
Wrote 34_Denise_Richards_0009.jpg
34_Denise_Richards_0010.jpg
Wrote 34_Denise_Richards_0010.jpg
34_Denise_Richards_0011.jpg
Wrot

Wrote 34_Elizabeth_Banks_0004.jpg
34_Elizabeth_Banks_0005.jpg
Wrote 34_Elizabeth_Banks_0005.jpg
34_Elizabeth_Banks_0006.jpg
Wrote 34_Elizabeth_Banks_0006.jpg
34_Elizabeth_Banks_0007.jpg
Wrote 34_Elizabeth_Banks_0007.jpg
34_Elizabeth_Banks_0008.jpg
Wrote 34_Elizabeth_Banks_0008.jpg
34_Elizabeth_Banks_0011.jpg
Wrote 34_Elizabeth_Banks_0011.jpg
34_Elizabeth_Banks_0012.jpg
Wrote 34_Elizabeth_Banks_0012.jpg
34_Elizabeth_Banks_0013.jpg
Wrote 34_Elizabeth_Banks_0013.jpg
34_Elizabeth_Banks_0014.jpg
Wrote 34_Elizabeth_Banks_0014.jpg
34_Elizabeth_Banks_0015.jpg
Wrote 34_Elizabeth_Banks_0015.jpg
34_Elizabeth_Banks_0016.jpg
Wrote 34_Elizabeth_Banks_0016.jpg
34_Elizabeth_Berkley_0001.jpg
Wrote 34_Elizabeth_Berkley_0001.jpg
34_Elizabeth_Berkley_0002.jpg
Wrote 34_Elizabeth_Berkley_0002.jpg
34_Elizabeth_Berkley_0003.jpg
Wrote 34_Elizabeth_Berkley_0003.jpg
34_Elizabeth_Berkley_0004.jpg
Wrote 34_Elizabeth_Berkley_0004.jpg
34_Elizabeth_Berkley_0007.jpg
Wrote 34_Elizabeth_Berkley_0007.jpg
34_Elizabeth_Ber

Wrote 34_Evangeline_Lilly_0004.jpg
34_Evangeline_Lilly_0006.jpg
Wrote 34_Evangeline_Lilly_0006.jpg
34_Evangeline_Lilly_0007.jpg
Wrote 34_Evangeline_Lilly_0007.jpg
34_Evangeline_Lilly_0008.jpg
Wrote 34_Evangeline_Lilly_0008.jpg
34_Evangeline_Lilly_0009.jpg
Wrote 34_Evangeline_Lilly_0009.jpg
34_Evangeline_Lilly_0010.jpg
Wrote 34_Evangeline_Lilly_0010.jpg
34_Evangeline_Lilly_0011.jpg
Wrote 34_Evangeline_Lilly_0011.jpg
34_Evangeline_Lilly_0012.jpg
Wrote 34_Evangeline_Lilly_0012.jpg
34_Evangeline_Lilly_0013.jpg
Wrote 34_Evangeline_Lilly_0013.jpg
34_Evangeline_Lilly_0014.jpg
34_Evangeline_Lilly_0015.jpg
Wrote 34_Evangeline_Lilly_0015.jpg
34_Evangeline_Lilly_0016.jpg
Wrote 34_Evangeline_Lilly_0016.jpg
34_Eva_Longoria_0001.jpg
Wrote 34_Eva_Longoria_0001.jpg
34_Eva_Longoria_0002.jpg
Wrote 34_Eva_Longoria_0002.jpg
34_Eva_Longoria_0003.jpg
Wrote 34_Eva_Longoria_0003.jpg
34_Eva_Longoria_0005.jpg
Wrote 34_Eva_Longoria_0005.jpg
34_Eva_Longoria_0006.jpg
Wrote 34_Eva_Longoria_0006.jpg
34_Eva_Longoria_

Wrote 34_Grace_Park_0006.jpg
34_Grace_Park_0007.jpg
Wrote 34_Grace_Park_0007.jpg
34_Grace_Park_0008.jpg
Wrote 34_Grace_Park_0008.jpg
34_Grace_Park_0009.jpg
Wrote 34_Grace_Park_0009.jpg
34_Grace_Park_0011.jpg
Wrote 34_Grace_Park_0011.jpg
34_Grace_Park_0012.jpg
Wrote 34_Grace_Park_0012.jpg
34_Grace_Park_0013.jpg
Wrote 34_Grace_Park_0013.jpg
34_Grace_Park_0014.jpg
Wrote 34_Grace_Park_0014.jpg
34_Grace_Park_0015.jpg
Wrote 34_Grace_Park_0015.jpg
34_Gretchen_Mol_0001.jpg
Wrote 34_Gretchen_Mol_0001.jpg
34_Gretchen_Mol_0002.jpg
Wrote 34_Gretchen_Mol_0002.jpg
34_Gretchen_Mol_0003.jpg
Wrote 34_Gretchen_Mol_0003.jpg
34_Gretchen_Mol_0004.jpg
Wrote 34_Gretchen_Mol_0004.jpg
34_Gretchen_Mol_0006.jpg
Wrote 34_Gretchen_Mol_0006.jpg
34_Gretchen_Mol_0008.jpg
Wrote 34_Gretchen_Mol_0008.jpg
34_Gretchen_Mol_0011.jpg
Wrote 34_Gretchen_Mol_0011.jpg
34_Gretchen_Mol_0012.jpg
Wrote 34_Gretchen_Mol_0012.jpg
34_Gwyneth_Paltrow_0001.jpg
Wrote 34_Gwyneth_Paltrow_0001.jpg
34_Gwyneth_Paltrow_0002.jpg
Wrote 34_Gwyneth_

In [68]:
import glob
import shutil
files=glob.glob('./CACD2000.tar/CACD2000/*í*')      # List of Byte string file names
for file in files:
    shutil.move(file, './CACD2000.tar/deleted')  